In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
#pandas settings
settings= {'display.max_rows': 50,
           'display.max_columns': 200,
           'display.width': 1000}

for k, v in settings.items():
    pd.set_option(k,v)
    
#matplotlib settings
plt.style.use('ggplot')


In [3]:
#Load raw data to pandas
path= "../../Data/Raw_Data/"

raw_energy= pd.read_csv(path + "energy_dataset.csv")
raw_weather= pd.read_csv(path + "weather_features.csv")

In [4]:
#preview energy data
print("Energy shape: ", raw_energy.shape)
raw_energy.head()

Energy shape:  (35064, 29)


,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,1164.0,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,1503.0,949.0,779.0,0.0,7098.0,43.0,75.0,50.0,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,1826.0,953.0,720.0,0.0,7097.0,43.0,74.0,42.0,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [5]:
#preview weather data
print("Weather shape: ", raw_weather.shape)
raw_weather.head()

Weather shape:  (178396, 17)


,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n


Data Cleaning

In [6]:
#Import modules
path= "../../"
os.chdir(path)

In [9]:
from src.DataCleaning import clean_columns, set_time_index, kelvin_to_fahrenheit, input_mean
        
#Clean raw_energy
clean_energy= set_time_index(input_mean(clean_columns(raw_energy)), time_column= "time")

#Clean raw_weather
clean_weather= set_time_index(clean_columns(raw_weather, cols= None), time_column= "dt_iso")

weather_cols= ["temp", "temp_min", "temp_max"]
clean_weather= kelvin_to_fahrenheit(clean_weather, weather_cols)

/Users/ksharma/Documents/ML Engineer/Machine Learning/Projects/Python Sandbox/EnergyPricePredictor/src/DataCleaning.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataframe= dataframe.fillna(dataframe.mean())


In [10]:
#Sanity checks
print("Old energy: %s  New energy: %s" % (raw_energy.shape, clean_energy.shape))
print("Old weather: %s  New weather: %s" % (raw_weather.shape, clean_weather.shape))

Old energy: (35064, 29)  New energy: (35064, 17)
Old weather: (178396, 17)  New weather: (178396, 16)


Stats about energy data

In [ ]:
#Variable types
clean_energy.info()

In [ ]:
#Descriptive stats
clean_energy.describe().T

# EDA

TSO Prediction Metrics

In [11]:
#Get arrays of target and predictions
tso_preds= clean_energy.loc[:,"price day ahead"].values
y= clean_energy.loc[:,"price actual"].values

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

#MSE, RMSE, MAE, MAPE
mse= mean_squared_error(y, tso_preds)
rmse= np.sqrt(mse)
mae= mean_absolute_error(y, tso_preds)
mape= mean_absolute_percentage_error(y, tso_preds)

keys= ["mse", "rmse", "mae", "mape"]
values= [mse, rmse, mae, mape]
eval_metrics= {}
for k, v in zip(keys, values):
    eval_metrics[k]= round(v,3)
    
print(eval_metrics)

{'mse': 175.559, 'rmse': 13.25, 'mae': 10.485, 'mape': 0.2}


Visualizing tso predictions vs reality by year

In [ ]:
#Break time series into plots
years= [2015, 2016, 2017, 2018]

#Plot data
plt.figure(figsize=(10,20))
plt.subplots_adjust(hspace=0.5)
plt.suptitle("Evaluating Annual TSO Price Predictions", fontsize=18, y=0.93)

for n, y in enumerate(years):
    mask = clean_energy.index.year == int(y)
    include = clean_energy[mask]
    exclude = clean_energy[~mask]
    
    # add a new subplot iteratively
    ax = plt.subplot(4,1, n + 1)

    # filter df and plot ticker on the new subplot axis
    include["price actual"].plot(ax=ax, alpha=.7)
    include["price day ahead"].plot(ax=ax, alpha= .7)
    # chart formatting
    ax.set_title(str(y).upper())
    ax.legend(shadow= True, loc= "lower right", labels=["Truth", "TSO Pred"])
    ax.set_xlabel("")

Correlation Matrix

In [ ]:
#drop price day ahead col for corr matrix
temp= clean_energy.drop(["price day ahead"], axis= 1)

#Correlation matrix
f, ax= plt.subplots(figsize= (15,15))
corrMatrix = temp.corr()
sns.heatmap(corrMatrix, annot=True, linewidths= .2, cmap="coolwarm")
plt.show()

Correlation matrix w/Target

In [ ]:
#features with greatest correlation
corrMatrix["price actual"][:-1].sort_values().plot(kind= "barh");

Weather and target

In [ ]:
#fix trailing/leading whitespaces in cityname
clean_weather['city_name']= clean_weather['city_name'].apply(lambda x: x.strip())

In [ ]:
#Create list of dataframes of city_name temps to merge
d= []
for index, city in enumerate(clean_weather.city_name.unique()):
    temp= pd.DataFrame(data= clean_weather.loc[clean_weather["city_name"] == city, "temp"],)
    temp.columns= [city]
    d.append(temp)

In [ ]:
for i in range(len(d)):
    if i <= 1:
        temp_merge= pd.merge(d[i-1], d[i], how= "inner", left_index=True, right_index=True)
    else:
        temp_merge= pd.merge(temp_merge, d[i], how= "inner", left_index=True, right_index=True)
temp_merge

In [ ]:
final_df= pd.merge(clean_energy, temp_merge, how= "left", left_index=True, right_index=True)
final_df.head()

In [ ]:
#drop price day ahead col for corr matrix
temp= final_df.drop(["price day ahead"], axis= 1)

#Correlation matrix
f, ax= plt.subplots(figsize= (15,15))
corrMatrix = temp.corr()
sns.heatmap(corrMatrix, annot=True, linewidths= .2, cmap="coolwarm")
plt.show()

In [ ]:
#View Correlation matrix with temperature data added into dataset
corrMatrix = final_df.corr()

#sort and remove price actual/price day ahead cols
f, ax= plt.subplots(figsize= (12,5))
corrMatrix["price actual"].sort_values()[:-2].plot(kind= "barh");

Distribution of variables

In [ ]:
final_df.shape

In [ ]:
for col in final_df.columns:
    final_df[col].plot(kind= "hist")

In [ ]:
f, axs= plt.subplots(8,4, figsize = (20,40))
axs= axs.flatten()

for i, col in enumerate(final_df.columns):
    final_df[col].plot.kde(ax= axs[i], color= "blue",)
    axs[i].set_title(col)

plt.tight_layout()

In [ ]:
#Write final_df to csv
path= "../Data/"
final_df.to_csv(path + "CleanedWeatherEnergyData.csv",)